<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Impute-education-level-from-'edu_1989_rev'-to-'edu_2003_rev'" data-toc-modified-id="Impute-education-level-from-'edu_1989_rev'-to-'edu_2003_rev'-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Impute education level from 'edu_1989_rev' to 'edu_2003_rev'</a></span></li></ul></div>

# Mortality Cleaning cont'd 3- 
- for blank spaces to 99 
- Add ID columns for SQL




In [63]:
# Essentials
import sys
import pandas as pd
pd.set_option('display.max_columns', 100)
import csv
import numpy as np
import os

# Plots
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Stats
from scipy import stats
from scipy.stats import skew


In [64]:
%%time 
# Load all files
# m2018 = pd.read_csv('../data/rawdata/mort_2018.csv', low_memory=False)
m2017 = pd.read_csv('../data/rawdata/mort_2017.csv', low_memory=False)
# m2016 = pd.read_csv('data/rawdata/mort_2016.csv', low_memory=False)
# m2015 = pd.read_csv('data/rawdata/mort_2015.csv', low_memory=False)
# m2014 = pd.read_csv('data/rawdata/mort_2014.csv', low_memory=False)
# m2013 = pd.read_csv('data/rawdata/mort_2013.csv', low_memory=False)
# m2012 = pd.read_csv('data/rawdata/mort_2012.csv', low_memory=False)
# m2011 = pd.read_csv('data/rawdata/mort_2011.csv', low_memory=False)

Wall time: 2min 44s


In [12]:
m2018.sample(7)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
357101,357102,8,1,51,,5,1,10,M,1,091 442411,1091,,44,24,11,,1,M,6,2018,U,0,C,N,99,99,J690273 088 37,J690,273,88,,37,0411A419 21J690 61E889 62R53 ...,4,11A419 21J690 61E889 62R53 ...,11A419,21J690,61E889,62R53,,,,,,,,,,,,,,,,,4,J690 A419 E889 R53 ...,J690,A419,E889,R53,,,,,,,,,,,,,,,,,68 24,68,99,99,2,4,100,8
2642418,2642419,8,1,81,,8,1,4,M,1,060 381808,1060,,38,18,8,,4,D,9,2018,U,7,C,Y,99,99,I429227 068 22,I429,227,68,,22,0511I429 61I709 62I10 63F101 64F179 ...,5,11I429 61I709 62I10 63F101 64F179 ...,11I429,61I709,62I10,63F101,64F179,,,,,,,,,,,,,,,,5,I429 F101 F179 I10 I709 ...,I429,F101,F179,I10,I709,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1231848,1231849,8,1,71,,7,1,7,M,1,085 432311,1085,,43,23,11,,6,W,6,2018,U,7,C,N,99,99,J449267 086 28,J449,267,86,,28,0311J449 21I500 61F179 ...,3,11J449 21I500 61F179 ...,11J449,21I500,61F179,,,,,,,,,,,,,,,,,,3,J449 F179 I500 ...,J449,F179,I500,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
292596,292597,8,1,31,,3,1,7,F,1,084 422210,1084,,42,22,10,,4,M,2,2018,U,0,C,N,99,99,C56 109 032 10,C56,109,32,,10,0311R092 21C56 22C787 ...,3,11R092 21C56 22C787 ...,11R092,21C56,22C787,,,,,,,,,,,,,,,,,,3,C56 C787 R092 ...,C56,C787,R092,,,,,,,,,,,,,,,,,,01 211,1,99,2,1,1,231,4
1132354,1132355,8,1,31,,3,1,8,F,1,074 402009,1074,,40,20,9,,1,W,3,2018,U,7,B,N,99,99,I469228 068 22,I469,228,68,,22,0111I469 ...,1,11I469 ...,11I469,,,,,,,,,,,,,,,,,,,,1,I469 ...,I469,,,,,,,,,,,,,,,,,,,,02 32,2,99,99,3,2,100,7
304094,304095,8,1,31,,3,1,7,F,1,095 452511,1095,,45,25,11,,4,W,6,2018,U,0,C,N,99,99,I422227 068 22,I422,227,68,,22,0811I469 21I509 31I422 41E785 61I059 62I060 63...,8,11I469 21I509 31I422 41E785 61I059 62I060 63I0...,11I469,21I509,31I422,41E785,61I059,62I060,63I051,64M819,,,,,,,,,,,,,7,I422 E785 I051 I080 I469 I509 M819 ...,I422,E785,I051,I080,I469,I509,M819,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1770166,1770167,8,1,81,,8,1,1,M,1,079 412110,1079,,41,21,10,,1,M,2,2018,U,7,C,Y,99,99,N185327 100 31,N185,327,100,,31,0411N19 21N185 31Q613 61C259 ...,4,11N19 21N185 31Q613 61C259 ...,11N19,21N185,31Q613,61C259,,,,,,,,,,,,,,,,,3,N185 C259 Q613 ...,N185,C259,Q613,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [13]:
m2017.sample(7)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
222370,222371,7,1,41,,4,1,4,F,1,094 442411,1094,,44,24,11,,6,W,1,2017,U,0,B,N,99,99,E149159 046 16,E149,159,46,,16,0611I469 21I709 31E149 61I10 62D370 63I48 ...,6,11I469 21I709 31E149 61I10 62D370 63I48 ...,11I469,21I709,31E149,61I10,62D370,63I48,,,,,,,,,,,,,,,6,E149 D370 I10 I469 I48 I709 ...,E149,D370,I10,I469,I48,I709,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1033000,1033001,7,1,31,,3,1,8,M,1,051 361607,1051,,36,16,7,,4,W,5,2017,U,7,B,Y,99,99,I250214 062 21,I250,214,62,,21,0711I250 61I308 62L029 63I517 64F101 65J439 66...,7,11I250 61I308 62L029 63I517 64F101 65J439 66E1...,11I250,61I308,62L029,63I517,64F101,65J439,66E141,,,,,,,,,,,,,,7,I250 E141 F101 I308 I517 J439 L029 ...,I250,E141,F101,I308,I517,J439,L029,,,,,,,,,,,,,,01 211,1,99,2,1,1,210,1
1133146,1133147,7,1,11,,1,1,12,M,1,093 442411,1093,,44,24,11,,6,M,1,2017,U,7,B,N,99,99,G309189 052 17,G309,189,52,,17,0211G309 61C189 ...,2,11G309 61C189 ...,11G309,61C189,,,,,,,,,,,,,,,,,,,2,G309 C189 ...,G309,C189,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
652986,652987,7,2,21,,2,1,9,M,1,035 331306,1035,,33,13,6,,7,S,4,2017,Y,1,C,Y,9,9,W24 423 123 39,W24,423,123,,39,0211T149 12W24 ...,2,11T149 12W24 ...,11T149,12W24,,,,,,,,,,,,,,,,,,,2,W24 T149 ...,W24,T149,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,260,2
790699,790700,7,1,31,,3,1,3,M,1,097 452511,1097,,45,25,11,,6,W,7,2017,U,7,C,N,99,99,J449267 086 28,J449,267,86,,28,0211J449 61F179 ...,2,11J449 61F179 ...,11J449,61F179,,,,,,,,,,,,,,,,,,,2,J449 F179 ...,J449,F179,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1551656,1551657,7,1,31,,3,1,7,F,1,088 432311,1088,,43,23,11,,6,W,5,2017,U,7,B,N,99,99,C23 087 043 15,C23,87,43,,15,0111C23 ...,1,11C23 ...,11C23,,,,,,,,,,,,,,,,,,,,1,C23 ...,C23,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1437345,1437346,7,2,51,,5,1,11,M,1,070 402009,1070,,40,20,9,,6,S,5,2017,U,7,B,N,99,99,C259088 025 07,C259,88,25,,7,0111C259 ...,1,11C259 ...,11C259,,,,,,,,,,,,,,,,,,,,1,C259 ...,C259,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [11]:
# df.drop('column_name', axis=1, inplace=True)
m2018.drop('edu_1989_Desc', axis=1, inplace=True)
m2018.drop('edu_2003_Desc', axis=1, inplace=True)
m2017.drop('edu_1989_Desc', axis=1, inplace=True)
m2017.drop('edu_2003_Desc', axis=1, inplace=True)


In [6]:
m2011.head()

,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
0,2,12 0,12,,0,1,F,1,065 391909,1065,,39,19,9,,1,M,2,2011,U,7,U,N,,,C859129 039 13,C859,129,39,,13,0111C859 ...,1,11C859 ...,11C859,,,,,,,,,,,,,,,,,,,,1,C859 ...,C859,,,,,,,,,,,,,,,,,,,,01 11,1,,,1,1,100,6
1,1,00 0,0,,0,1,F,1,057 371708,1057,,37,17,8,,1,S,7,2011,U,7,U,N,,,C509104 029 09,C509,104,29,,9,0211R092 21C509 ...,2,11R092 21C509 ...,11R092,21C509,,,,,,,,,,,,,,,,,,,2,C509 R092 ...,C509,R092,,,,,,,,,,,,,,,,,,,03 23,3,,,2,3,100,8
2,1,14 0,14,,0,1,M,1,075 412110,1075,,41,21,10,,1,M,3,2011,U,7,U,N,,,I251215 063 21,I251,215,63,,21,0511K559 21I251 31I739 41J449 61N185 ...,5,11K559 21I251 31I739 41J449 61N185 ...,11K559,21I251,31I739,41J449,61N185,,,,,,,,,,,,,,,,5,I251 I739 J449 K559 N185 ...,I251,I739,J449,K559,N185,,,,,,,,,,,,,,,,01 11,1,,,1,1,100,6
3,1,16 0,16,,0,1,F,1,061 381808,1061,,38,18,8,,4,M,1,2011,U,7,U,N,,,I250214 062 21,I250,214,62,,21,0311I250 61F102 62J449 ...,3,11I250 61F102 62J449 ...,11I250,61F102,62J449,,,,,,,,,,,,,,,,,,3,I250 F102 J449 ...,I250,F102,J449,,,,,,,,,,,,,,,,,,01 11,1,,,1,1,100,6
4,1,12 0,12,,0,1,M,1,020 301004,1020,,30,10,4,,7,S,7,2011,N,1,U,N,9,,V869396 114 38,V869,396,114,,38,0211S099 12V869 ...,2,11S099 12V869 ...,11S099,12V869,,,,,,,,,,,,,,,,,,,2,V869 S099 ...,V869,S099,,,,,,,,,,,,,,,,,,,03 23,3,,,2,3,100,8


In [24]:
%%time
# replace blanks in activity_code with 99, change dtype to int64
# m2018['activity_code'] = m2018['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2018 activity_code complete')
m2017['activity_code'] = m2017['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2017 activity_code complete')
# m2016['activity_code'] = m2016['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2016 activity_code complete')
# m2015['activity_code'] = m2015['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2015 activity_code complete')
# m2014['activity_code'] = m2014['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2014 activity_code complete')
# m2013['activity_code'] = m2013['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2013 activity_code complete')
# m2012['activity_code'] = m2012['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2012 activity_code complete')
# m2011['activity_code'] = m2011['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2011 activity_code complete')

Wall time: 1min 7s


In [8]:
m2011['place_of_injury'].dtype

dtype('O')

In [25]:
%%time
# replace blanks in place_of_injury with 99, change dtype to int64
# m2018['place_of_injury'] = m2018['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2018 place_of_injury complete')
m2017['place_of_injury'] = m2017['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
print('m2017 place_of_injury complete')
# m2016['place_of_injury'] = m2016['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2016 place_of_injury complete')
# m2015['place_of_injury'] = m2015['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2015 place_of_injury complete')
# m2014['place_of_injury'] = m2014['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2014 place_of_injury complete')
# m2013['place_of_injury'] = m2013['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2013 place_of_injury complete')
# m2012['place_of_injury'] = m2012['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2012 place_of_injury complete')
# m2011['place_of_injury'] = m2011['place_of_injury'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2011 place_of_injury complete')

m2017 place_of_injury complete
Wall time: 1min 12s


In [26]:
%%time
# replace blanks in bridged_race_flag, race_imputation_flag with 99, change dtype to int64
# m2018[['bridged_race_flag', 'race_imputation_flag']] = m2018[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2018 bridged_race_flag, race_imputation_flag complete')

%time m2017[['bridged_race_flag', 'race_imputation_flag']] = m2017[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
print('m2017 bridged_race_flag, race_imputation_flag complete')
# m2016[['bridged_race_flag', 'race_imputation_flag']] = m2016[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2016 bridged_race_flag, race_imputation_flag complete')

# %time m2015[['bridged_race_flag', 'race_imputation_flag']] = m2015[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2015 bridged_race_flag, race_imputation_flag complete')
# m2014[['bridged_race_flag', 'race_imputation_flag']] = m2014[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2014 bridged_race_flag, race_imputation_flag complete')

# %time m2013[['bridged_race_flag', 'race_imputation_flag']] = m2013[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2013 bridged_race_flag, race_imputation_flag complete')
# m2012[['bridged_race_flag', 'race_imputation_flag']] = m2012[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2012 bridged_race_flag, race_imputation_flag complete')

# %time m2011[['bridged_race_flag', 'race_imputation_flag']] = m2011[['bridged_race_flag', 'race_imputation_flag']].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')
# print('m2011 bridged_race_flag, race_imputation_flag complete')

Wall time: 10.7 s
m2017 bridged_race_flag, race_imputation_flag complete
Wall time: 10.7 s


In [263]:
# dfs = list('m2011', 'm2012', 'm2013', 'm2014', 'm2015', 'm2016', 'm2017', 'm2018')
# for df in dfs:
#     if (df['activity_code'].dtype == dtype('O')):
#         df['activity_code'] = df['activity_code'].apply(pd.to_numeric, errors='coerce').fillna(0).astype('int64')
#         print(df['activity_code'].dtype)
#     else:
#         continue


In [27]:
# df.drop('column_name', axis=1, inplace=True)
m2017.drop('year_id', axis=1, inplace=True)
m2017.head()

,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
0,1,41,,4,1,1,M,1,071 402009,1071,,40,20,9,,1,M,2,2017,U,7,C,N,99,99,G20 188 051 37,G20,188,51,,37,0111G20 ...,1,11G20 ...,11G20,,,,,,,,,,,,,,,,,,,,1,G20 ...,G20,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1,1,11,,1,1,1,M,1,074 402009,1074,,40,20,9,,1,D,4,2017,U,7,C,N,99,99,M726315 111 37,M726,315,111,,37,0211A419 21M726 ...,2,11A419 21M726 ...,11A419,21M726,,,,,,,,,,,,,,,,,,,2,M726 A419 ...,M726,A419,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
2,1,41,,4,1,1,M,1,059 371708,1059,,37,17,8,,4,M,3,2017,U,7,C,N,99,99,C159077 021 15,C159,77,21,,15,0211C159 61F179 ...,2,11C159 61F179 ...,11C159,61F179,,,,,,,,,,,,,,,,,,,2,C159 F179 ...,C159,F179,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
3,1,11,,1,1,1,M,1,090 442411,1090,,44,24,11,,4,M,3,2017,U,7,B,N,99,99,C61 113 033 11,C61,113,33,,11,0111C61 ...,1,11C61 ...,11C61,,,,,,,,,,,,,,,,,,,,1,C61 ...,C61,,,,,,,,,,,,,,,,,,,,03 23,3,99,99,2,3,100,8
4,1,51,,5,1,1,M,1,066 391909,1066,,39,19,9,,4,M,3,2017,U,7,C,N,99,99,J449267 086 28,J449,267,86,,28,0311J449 61C900 62F179 ...,3,11J449 61C900 62F179 ...,11J449,61C900,62F179,,,,,,,,,,,,,,,,,,3,J449 C900 F179 ...,J449,C900,F179,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [28]:
m2018.sample(3)

,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
1847412,8,1,11,,1,1,10,M,1,097 452511,1097,,45,25,11,,6,W,1,2018,U,7,E,N,99,99,F03 175 111 37,F03,175,111,,37,0211F03 21N189 ...,2,11F03 21N189 ...,11F03,21N189,,,,,,,,,,,,,,,,,,,2,F03 N189 ...,F03,N189,,,,,,,,,,,,,,,,,,,04 24,4,99,99,2,4,100,8
998084,8,1,31,,3,1,8,M,1,053 361607,1053,,36,16,7,,4,M,1,2018,U,7,C,N,99,99,C64 117 034 12,C64,117,34,,12,0111C64 ...,1,11C64 ...,11C64,,,,,,,,,,,,,,,,,,,,1,C64 ...,C64,,,,,,,,,,,,,,,,,,,,02 32,2,99,99,3,2,100,7
926159,8,1,11,,1,1,9,M,1,080 422210,1080,,42,22,10,,4,M,1,2018,U,7,B,N,99,99,C833129 039 13,C833,129,39,,13,0111C833 ...,1,11C833 ...,11C833,,,,,,,,,,,,,,,,,,,,1,C833 ...,C833,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,235,4


In [29]:
%%time
m2018.insert(0, 'year_id', '8')
m2017.insert(0, 'year_id', '7')
m2016.insert(0, 'year_id', '6')
m2015.insert(0, 'year_id', '5')
m2014.insert(0, 'year_id', '4')
m2013.insert(0, 'year_id', '3')
m2012.insert(0, 'year_id', '2')
m2011.insert(0, 'year_id', '1')

In [30]:
%%time
m2018.insert(0, 'person_id', np.arange(1,len(m2018)+1,1))
m2017.insert(0, 'person_id', np.arange(1,len(m2017)+1,1))
m2016.insert(0, 'person_id', np.arange(1,len(m2016)+1,1))
m2015.insert(0, 'person_id', np.arange(1,len(m2015)+1,1))
m2014.insert(0, 'person_id', np.arange(1,len(m2014)+1,1))
m2013.insert(0, 'person_id', np.arange(1,len(m2013)+1,1))
m2012.insert(0, 'person_id', np.arange(1,len(m2012)+1,1))
m2011.insert(0, 'person_id', np.arange(1,len(m2011)+1,1))

Wall time: 154 ms


In [35]:
m2011.sample(5)

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
1232037,1232038,1,1,41,,4,1,8,M,1,082 422210,1082,,42,22,10,,4,M,7,2011,U,7,B,N,99,99,C349093 027 08,C349,93,27,,8,0411C349 21J449 61F101 62F179 ...,4,11C349 21J449 61F101 62F179 ...,11C349,21J449,61F101,62F179,,,,,,,,,,,,,,,,,4,C349 F101 F179 J449 ...,C349,F101,F179,J449,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1201508,1201509,1,1,31,,3,1,9,M,1,067 391909,1067,,39,19,9,,6,M,1,2011,U,7,C,N,99,99,J449267 086 28,J449,267,86,,28,0411J449 21I500 31I10 61F179 ...,4,11J449 21I500 31I10 61F179 ...,11J449,21I500,31I10,61F179,,,,,,,,,,,,,,,,,4,J449 F179 I10 I500 ...,J449,F179,I10,I500,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1894354,1894355,1,1,15 0,15,,0,6,F,1,076 412110,1076,,41,21,10,,1,W,1,2011,U,7,U,U,99,99,K729301 111 37,K729,301,111,,37,0211N179 21K729 ...,2,11N179 21K729 ...,11N179,21K729,,,,,,,,,,,,,,,,,,,2,K729 N179 ...,K729,N179,,,,,,,,,,,,,,,,,,,02 32,2,99,99,3,2,100,7
2244441,2244442,1,2,61,,6,1,12,F,1,070 402009,1070,,40,20,9,,1,M,6,2011,U,7,C,N,99,99,C56 109 032 10,C56,109,32,,10,0211C56 12C787 ...,2,11C56 12C787 ...,11C56,12C787,,,,,,,,,,,,,,,,,,,2,C56 C787 ...,C56,C787,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
259050,259051,1,1,31,,3,1,7,F,1,094 442411,1094,,44,24,11,,4,W,5,2011,U,0,B,N,99,99,F03 175 111 37,F03,175,111,,37,0111F03 ...,1,11F03 ...,11F03,,,,,,,,,,,,,,,,,,,,1,F03 ...,F03,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6


In [44]:
m2011['edu_2003_rev'].dtype

dtype('O')

In [45]:
m2011.groupby('edu_2003_rev').size()

edu_2003_rev
     674312
1    233153
2    204862
3    765032
4    224101
5     97110
6    174626
7     66149
8     29473
9     46640
dtype: int64

In [46]:
edu_1989_dtl=pd.read_excel('./keys/edu_1989_rev_LK.xlsx' )
edu_2003_dtl=pd.read_excel('./keys/edu_2003_rev_LK.xlsx' )

In [47]:
edu_1989_dtl.sample(3)

,edu_1989_rev,edu_1989_Desc
16,16,BA BS
7,7,1-8 Grd
12,12,HsGedCom


In [52]:
edu_1989_dtl['edu_1989_Desc'].apply(len)

0     8
1     7
2     7
3     7
4     7
5     7
6     7
7     7
8     7
9     7
10    7
11    7
12    8
13    7
14    7
15    7
16    5
17    6
18    6
Name: edu_1989_Desc, dtype: int64

In [81]:
edu_2003_dtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   edu_2003_rev   9 non-null      int64 
 1   edu_2003_Desc  9 non-null      object
dtypes: int64(1), object(1)
memory usage: 272.0+ bytes


In [75]:
m2017['edu_2003_rev'].dtype

dtype('O')

In [85]:
edu_2003_dtl['edu_2003_rev'].dtype

dtype('int64')

In [53]:
edu_2003_dtl['edu_2003_Desc'].apply(len)

0    7
1    7
2    8
3    8
4    5
5    5
6    7
7    8
8    6
Name: edu_2003_Desc, dtype: int64

In [54]:
edu_1989_dtl["edu_1989_rev"] = edu_1989_dtl["edu_1989_rev"].apply(str)
edu_1989_dtl["edu_1989_rev"] = edu_1989_dtl["edu_1989_rev"].str.zfill(2)

In [67]:
%%time
# m2018_t = pd.merge(m2018, edu_1989_dtl,on='edu_1989_rev',how='left')
m2017_t = pd.merge(m2017, edu_1989_dtl,on='edu_1989_rev',how='left')


In [62]:
m2017_t.sample(10)

NameError: name 'm2017_t' is not defined

In [94]:
edu_2003_dtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   edu_2003_rev   9 non-null      object
 1   edu_2003_Desc  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


In [93]:
# m2017_t['edu_2003_rev'].to_numeric

edu_2003_dtl['edu_2003_rev'] = edu_2003_dtl['edu_2003_rev'].apply(str)

In [95]:
%%time
# m2018_t = pd.merge(m2018_t, edu_2003_dtl,on='edu_2003_rev',how='left')
# m2017_t = pd.merge(m2017_t, edu_2003_dtl,on='edu_2003_rev',how='left')
m2017_t = pd.merge(m2017_t, edu_2003_dtl,on='edu_2003_rev',how='left')

Wall time: 1min 32s


### Impute education level from 'edu_1989_rev' to 'edu_2003_rev'
    1. change blank spaces in 'edu_1989_rev' to 99, assign type 'int64'
    2. change blank spaces in 'edu_2003_rev' to 0, assign to type 'int64'
    3. iterate for 0 in 'edu_2003_rev' look at value of 'edu_1989_rev' for data
        - if gr <9 gr = 1
        - if gr >=9 & <12 gr = 2
        - if gr ==12 gr = 3
        - if gr >12 & <16 gr = 4
        - if gr ==16 gr = 6
        - if gr ==17 gr = 7
        - if gr ==99 gr = 9
    

In [68]:
m2017.sample(20).sort_index()

,person_id,year_id,resident_status,edu,edu_1989_rev,edu_2003_rev,edu_reporting_flag,month_of_death,sex,reported_age_tmstmp_flag,reported_age,age_detail,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death_and_status,marital_status,day_of_week_of_death,data_year,injury_at_work,manner_of_death,method_of_disposition,autopsy,activity_code,place_of_injury,underlying_cause_of_death,icd_10th_revision,cause_recode_358,cause_recode_113,infant_cause_130,cause_recode_39,multiple_conditions,number_of_entityaxis_conditions,entityaxis_conditions_con,entAx_condition_1st,entAx_condition_2nd,entAx_condition_3rd,entAx_condition_4th,entAx_condition_5th,entAx_condition_6th,entAx_condition_7th,entAx_condition_8th,entAx_condition_9th,entAx_condition_10th,entAx_condition_11th,entAx_condition_12th,entAx_condition_13th,entAx_condition_14th,entAx_condition_15th,entAx_condition_16th,entAx_condition_17th,entAx_condition_18th,entAx_condition_19th,entAx_condition_20th,number_of_recordaxis_conditions,recordaxis_conditions,recAx_condition_1st,recAx_condition_2nd,recAx_condition_3rd,recAx_condition_4th,recAx_condition_5th,recAx_condition_6th,recAx_condition_7th,recAx_condition_8th,recAx_condition_9th,recAx_condition_10th,recAx_condition_11th,recAx_condition_12th,recAx_condition_13th,recAx_condition_14th,recAx_condition_15th,recAx_condition_16th,recAx_condition_17th,recAx_condition_18th,recAx_condition_19th,recAx_condition_20th,bridged_race,bridged_race_1,bridged_race_flag,race_imputation_flag,allother_races_imputed_2,bridged_race_recode_5,hispanicorigin,hispanicorigin_bridgeracerecode
291031,291032,7,1,21,,2,1,7,M,1,058 371708,1058,,37,17,8,,4,M,3,2017,U,7,B,Y,99,99,I250214 062 21,I250,214,62,,21,0311I119 12I250 61F179 ...,3,11I119 12I250 61F179 ...,11I119,12I250,61F179,,,,,,,,,,,,,,,,,,3,I250 F179 I119 ...,I250,F179,I119,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,210,1
419531,419532,7,1,41,,4,1,2,F,1,092 442411,1092,,44,24,11,,6,W,4,2017,U,7,B,N,99,99,G309189 052 17,G309,189,52,,17,0111G309 ...,1,11G309 ...,11G309,,,,,,,,,,,,,,,,,,,,1,G309 ...,G309,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
518711,518712,7,1,61,,6,1,2,M,1,079 412110,1079,,41,21,10,,1,M,6,2017,U,7,E,N,99,99,R688380 110 36,R688,380,110,,36,0211R688 12R54 ...,2,11R688 12R54 ...,11R688,12R54,,,,,,,,,,,,,,,,,,,2,R688 R54 ...,R688,R54,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
593756,593757,7,2,31,,3,1,6,M,1,062 381808,1062,,38,18,8,,1,M,4,2017,U,7,C,N,99,99,I219211 059 21,I219,211,59,,21,0411I501 21I219 31I500 41I710 ...,4,11I501 21I219 31I500 41I710 ...,11I501,21I219,31I500,41I710,,,,,,,,,,,,,,,,,4,I219 I500 I501 I710 ...,I219,I500,I501,I710,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
631539,631540,7,1,31,,3,1,8,M,1,063 381808,1063,,38,18,8,,5,M,5,2017,U,7,C,N,99,99,C900132 041 15,C900,132,41,,15,0111C900 ...,1,11C900 ...,11C900,,,,,,,,,,,,,,,,,,,,1,C900 ...,C900,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
997796,997797,7,2,31,,3,1,9,F,1,077 412110,1077,,41,21,10,,1,D,5,2017,U,7,B,N,99,99,I48 228 068 22,I48,228,68,,22,0511A419 21N179 31K720 41I48 61F179 ...,5,11A419 21N179 31K720 41I48 61F179 ...,11A419,21N179,31K720,41I48,61F179,,,,,,,,,,,,,,,,5,I48 A419 F179 K720 N179 ...,I48,A419,F179,K720,N179,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1048413,1048414,7,1,51,,5,1,2,F,1,097 452511,1097,,45,25,11,,1,W,4,2017,U,7,C,N,99,99,R092380 110 36,R092,380,110,,36,0111R092 ...,1,11R092 ...,11R092,,,,,,,,,,,,,,,,,,,,1,R092 ...,R092,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1247520,1247521,7,1,71,,7,1,2,M,1,092 442411,1092,,44,24,11,,4,W,5,2017,U,7,O,N,99,99,I251215 063 21,I251,215,63,,21,0111I251 ...,1,11I251 ...,11I251,,,,,,,,,,,,,,,,,,,,1,I251 ...,I251,,,,,,,,,,,,,,,,,,,,01 11,1,99,99,1,1,100,6
1294961,1294962,7,1,41,,4,1,3,M,1,057 371708,1057,,37,17,8,,2,M,5,2017,U,7,C,N,99,99,N185327 100 31,N185,327,100,,31,0211N185 61J189 ...,2,11N185 61J189 ...,11N185,61J189,,,,,,,,,,,,,,,,,,,2,N185 J189 ...,N185,J189,,,,,,,,,,,,,,,,,,,02 32,2,99,99,3,2,100,7
1317528,1317529,7,1,31,,3,1,7,F,1,07

In [53]:
m2017_edu = m2017[['person_id', 'edu_1989_rev', 'edu_2003_rev']].copy()

m2017_edu

,person_id,edu_1989_rev,edu_2003_rev
0,1,,4
1,2,,1
2,3,,4
3,4,,1
4,5,,5
...,...,...,...
2813498,2813499,,3
2813499,2813500,,3
2813500,2813501,,4
2813501,2813502,,1


In [54]:
print(m2017_edu['edu_1989_rev'].dtype)
print(m2017_edu['edu_1989_rev'].unique())
m2017_edu['edu_1989_rev'].value_counts()

object
['  ' '12' '14' '15' '16' '11' '07' '06' '99' '08' '09' '10' '00' '17'
 '13' '04' '05' '03' '02' '01' '9' '8' '7' '0' '6' '3' '5' '1' '4' '2']


      2798552
12       7482
14       1126
16        870
10        832
8         780
11        658
17        554
13        518
08        389
9         381
99        202
6         181
09        172
15        152
7         137
0         106
06         88
07         68
00         45
3          41
5          41
4          38
04         25
05         24
03         19
2          11
02          4
1           4
01          3
Name: edu_1989_rev, dtype: int64

In [58]:
# replace 'edu_1989_rev' blanks with 99 (Unknown), assign type int64
m2017_edu['edu_1989_rev'] = m2017_edu['edu_1989_rev'].apply(pd.to_numeric, errors='coerce').fillna(99).astype('int64')

# replace blanks with 0, assign type int64
m2017_edu['edu_2003_rev'] = m2017_edu['edu_2003_rev'].apply(pd.to_numeric, errors='coerce').fillna(0).astype('int64')


In [60]:
print(m2017_edu['edu_1989_rev'].dtype)
print(m2017_edu['edu_1989_rev'].unique())
m2017_edu['edu_1989_rev'].value_counts()

int64
[99 12 14 15 16 11  7  6  8  9 10  0 17 13  4  5  3  2  1]


99    2798754
12       7482
8        1169
14       1126
16        870
10        832
11        658
17        554
9         553
13        518
6         269
7         205
15        152
0         151
5          65
4          63
3          60
2          15
1           7
Name: edu_1989_rev, dtype: int64

In [59]:
print(m2017_edu['edu_2003_rev'].dtype)
print(m2017_edu['edu_2003_rev'].unique())
m2017_edu['edu_2003_rev'].value_counts()

int64
[4 1 5 3 6 9 2 7 8 0]


3    1184479
4     349299
6     293834
2     292239
1     285143
5     174805
7     120144
8      50485
9      48124
0      14951
Name: edu_2003_rev, dtype: int64

In [43]:
print(m2017_edu['edu_2003_rev'].dtype)
print(m2017_edu['edu_2003_rev'].unique())
m2017_edu['edu_2003_rev'].value_counts()

int64
[4 1 5 3 6 9 2 7 8 0]


3    1184479
4     349299
6     293834
2     292239
1     285143
5     174805
7     120144
8      50485
9      48124
0      14951
Name: edu_2003_rev, dtype: int64

In [61]:
# impute values from edu_1989_rev into 0 spaces of edu_2003_rev
def gr_imp(gr):
    if gr <9:
        new_grade = 1
    elif gr >=9 & gr <12:
        new_grade = 2
    elif gr ==12:
        new_grade = 3
    elif gr >12 & gr <16:
        new_grade = 4
    elif gr ==16:
        new_grade = 6
    elif gr ==17:
        new_grade = 7
    else:
        new_grade = 9
    return new_grade
        


In [97]:
# m2018.insert(0, 'year_id', '8')
# m2018.insert(5, 'edu_1989_Desc', m2018_t['edu_1989_Desc'])
m2017.insert(5, 'edu_1989_Desc', m2017_t['edu_1989_Desc'])


In [98]:
# m2018.insert(7, 'edu_2003_Desc', m2018_t['edu_2003_Desc'])
m2017.insert(7, 'edu_2003_Desc', m2017_t['edu_2003_Desc'])

In [14]:
%%time
m2018.to_csv('../data/rawdata/mort_2018.csv', index = False, header = True)
print('m2018 file complete')
m2017.to_csv('../data/rawdata/mort_2017.csv', index = False, header = True)
print('m2017 file complete')
# m2016.to_csv('data/rawdata/mort_2016.csv', index = False, header = True)
# print('m2016 file complete')
# m2015.to_csv('data/rawdata/mort_2015.csv', index = False, header = True)
# print('m2015 file complete')
# m2014.to_csv('data/rawdata/mort_2014.csv', index = False, header = True)
# print('m2014 file complete')
# m2013.to_csv('data/rawdata/mort_2013.csv', index = False, header = True)
# print('m2013 file complete')
# m2012.to_csv('data/rawdata/mort_2012.csv', index = False, header = True)
# print('m2012 file complete')
# m2011.to_csv('data/rawdata/mort_2011.csv', index = False, header = True)
# print('m2011 file complete')

m2018 file complete
m2017 file complete
Wall time: 4min 53s
